### 5.2.1 모델 개요

[Doc2Vec](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)은 구글 연구 팀이 개발한 문서 임베딩 기법이다. 

일반적인 언어 모델(단어 시퀀스 $k$개가 주어졌을 때 그 다음 단어를 맞추는 모델)은 다음과 같이 학습된다. 학습 데이터 문장 하나의 단어 개수가 $T$일 때 해당 문장의 로그 확률의 평균은 다음과 같다.
$$
\mathfrak{L} = \frac 1T \sum^{T}_{t=k} \log p(w_t|w_{t-k}, \ldots, w_{t-1})
$$

여기서 확률 p는 다음과 같이 정의된다.

$$
p(w_t|w_{t-k}, \ldots, w_{t-1}) = \frac {\exp(\mathbf{y}_{w_t})}{\sum_i \exp(\mathbf{y}_i)} \\
\mathbf{y} = \mathbf{b} + \mathbf{U}\cdot h(w_{t-k}, \ldots, w_{t-1};\mathbf{W})
$$

여기서 $\mathbf{y}_i$는 $i$번째 단어에 해당하는 점수, $\mathbf{W}$는 단어 행렬이며, $h$는 벡터 시퀀스가 주어졌을 때 평균을 취하거나 이어 붙여 고정된 길이의 벡터 하나를 반환하는 역할을 하는 함수다.

<img src='https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/doc2vec.png?fit=1200%2C667' width=400>

Doc2Vec은 여기에 추가로 문서ID를 고려하는 언어 모델이다. $\mathbf{y}$를 계산할 때 $\mathbf{D}$라는 문서 행렬(paragraph matrix)에서 해당 문서 ID에 대항하는 벡터를 참조해 $h$에 다른 단어 벡터들과 함께 입력하는 것이다. 저자들은 이 모델을 PV-DM(the Distributed Memory Model of Paragraph Vectors)이라고 이름 붙였다. 학습이 종료되면 문서 수 * 임베딩 차원 수 크기를 가지는 문서 행렬 $\mathbf{D}$를 각 문서의 임베딩으로 사용한다.

<img src='https://miro.medium.com/max/552/1*2n35mvEFvym_eC_9XUdsTg.png' width = 400>

CBOW, Skip-gram의 관계와 유사하게, PV_DBOW라는 모델은 문서 ID를 가지고 문맥 단어를 학습하는 모델이다.

### 5.2.2 튜토리얼

네이버 영화 평점 데이터(`data/processed/processed_review_movieid.txt`)를 이용하여 Doc2Vec 모델을 학습시켜 보자. 해당 파일은 [코멘트,영화ID] 식으로 구성되어 있다.

In [5]:
data_path = 'data/processed/processed_review_movieid.txt'
corpus = [sent for sent in open(data_path, 'r')]
corpus[:3]

['종합 평점은 4점 드립니다.␞92575\n',
 '원작이 칭송받는 이유는 웹툰 계 자체의 질적 저하가 심각하기 때문.  원작이나 영화나 별로인건 마찬가지.␞92575\n',
 '나름의  감동도 있고 안타까운 마음에 가슴도 먹먹  배우들의 연기가 good 김수현 최고~␞92575\n']

In [8]:
corpus = [sent.replace('\n','').split('\u241E') for sent in open(data_path, 'r')]
corpus[:3]

[['종합 평점은 4점 드립니다.', '92575'],
 ['원작이 칭송받는 이유는 웹툰 계 자체의 질적 저하가 심각하기 때문.  원작이나 영화나 별로인건 마찬가지.', '92575'],
 ['나름의  감동도 있고 안타까운 마음에 가슴도 먹먹  배우들의 연기가 good 김수현 최고~', '92575']]

Doc2Vec 모델 학습을 위해 gensim 라이브러리의 Doc2Vec 클래스를 사용하자. 이때 Doc2VecInput은 이 클래스가 요구하는 입력 형태를 맞춰주는 역할을 한다.

In [11]:
from preprocess import get_tokenizer
from gensim.models.doc2vec import TaggedDocument

class Doc2VecInput:
    
    def __init__(self, fname, tokenizer_name='mecab'):
        self.fname = fname
        self.tokenizer = get_tokenizer(tokenizer_name)
        
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for line in f:
                try:
                    sentence, movie_id = line.strip().split('\u241E')
                    tokens = self.tokenizer.morphs(sentence)
                    tagged_doc = TaggedDocument(words=tokens, tags=['MOVIE_%s' % movie_id])
                    yield tagged_doc
                except:
                    continue

In [12]:
from gensim.models import Doc2Vec

corpus_fname = 'data/processed/processed_review_movieid.txt'
output_fname = 'data/sentence-embeddings/doc2vec/doc2vec.model'
corpus = Doc2VecInput(corpus_fname)

In [15]:
next(iter(corpus))

Exception ignored in: <generator object Doc2VecInput.__iter__ at 0x7efb9e635570>
RuntimeError: generator ignored GeneratorExit


TaggedDocument(words=['종합', '평점', '은', '4', '점', '드립니다', '.'], tags=['MOVIE_92575'])

다음과 같이 모델을 학습시키자. 여기서 dm은 0이면 PV-DBOW, 1이면 PV_DM 이다. 대충 5분 정도 걸림.

In [19]:
model = Doc2Vec(corpus, dm=1, vector_size=100)
model.save(output_fname)

In [27]:
model.docvecs.most_similar('MOVIE_36843')

[('MOVIE_19532', 0.7529395818710327),
 ('MOVIE_56607', 0.7330312728881836),
 ('MOVIE_28783', 0.7287821769714355),
 ('MOVIE_54260', 0.7275999784469604),
 ('MOVIE_11501', 0.7266227602958679),
 ('MOVIE_40208', 0.7264329791069031),
 ('MOVIE_28561', 0.7254093885421753),
 ('MOVIE_86749', 0.7227391600608826),
 ('MOVIE_17707', 0.7184653282165527),
 ('MOVIE_27321', 0.7179006338119507)]

책에 나온 방법대로 모델을 평가해보자.

In [20]:
from models.sent_eval import Doc2VecEvaluator
model_eval = Doc2VecEvaluator(output_fname)

학습 데이터에는 영화 제목이 없기 때문에 아래 명령은 네이버 영화 사이트(https://movie.naver.com)에 접속해 해당 영화 ID에 맞는 제목을 스크래핑 해온다.

In [21]:
model_eval.get_titles_in_corpus(n_sample=3)

{'MOVIE_25392': '아스테릭스', 'MOVIE_25754': '성항기병', 'MOVIE_70563': '우리 이웃의 범죄'}

In [25]:
# 36843 : 러브 액츄얼리
model_eval.most_similar(36843, topn=3)

크리스마스에 눈이 내리면 0.7529395818710327
호랑이와 눈 0.7330312728881836
우리의 사랑 0.7287821769714355


In [29]:
model_eval.visualize_movies(type='between')

save @ between-words.png


In [30]:
model_eval.visualize_movies(type='tsne')

save @ words.png


<img src='between-words.png'>

<img src='words.png'>